# KUIS UTS!!! 
Laman ini adalah merupakan file Jupyter Notebook untuk menggenerate dataset yang akan  digunakan untuk kuis di bawah ini. Hasil download file yang digenerate akan berbeda antar mahasiswa yang satu dengan yang lain. Berikut langkah untuk mendapatkan filesnya.

1. Copy terlebih dahulu file jupyter notebook ini ke google drive kalian dengan cara klik tombol 'Copy to Drive' di atas.

2. Setelah kalian klik, kalian akan diarahkan ke Tab baru. Kemudian, silahkan masukkan NPM anda di data dibawah ini (ganti nomor dibawah dengan NPM anda)

3. Setelah memasukkan NPM anda, silahkan klik menu 'Runtime' di atas, kemudian pilih 'Run All' atau bisa dengan tombol Ctrl+F9.

4. File "soal\_untuk\_[nomor npm]\_validasi.xlsx" dan "soal\_untuk\_[nomor npm]\_simulasi_sejajar.xlsx" akan terdownload secara otomatis (berikan waktu 10-30 detik). Apabila ada notifikasi, klik tombol 'Allow'.

5. Setiap mahasiswa akan mendapatkan data yang berbeda.

In [ ]:
npm_saya = 1302191504

## Introduction

Anda sedang mengaudit sebuah perusahaan. Perusahaan tersebut memberikan laporan kepada Anda dalam bentuk Excel. Sebelum melakukan pengujian, Anda melakukan validasi data dari file excel tersebut, baru setelah itu melakukan simulasi sejajar. File untuk validasi dan file untuk simulasi sejajar berbeda. Silahkan cek petunjuk di bawah ini.

## Tugas!!

A. Lakukan validasi data menggunakan file "soal\_untuk\_[nomor npm]\_validasi.xlsx" dengan kemungkinan kesalahan sebagai berikut:
  1. otorisator (id_pegawai) yang tidak berwenang dalam transaksi penjualan
  2. kode penjualan yang double
  3. kesalahan penulisan kode penjualan
  4. kesalahan periode pada tabel penjualan
  5. data kosong
  6. kode barang tidak sesuai
  7. adanya nilai minus
  8. kesalahan harga barang
  9. perhitungan matematis yang salah

B. Lakukan Simulasi Sejajar menggunakan file "soal\_untuk\_[nomor npm]\_simulasi_sejajar.xlsx" untuk 5 (lima) sheet laporan dari file , yaitu:
  1. Simulasi sejajar untuk Laporan Penjualan
  2. Simulasi sejajar untuk Laporan Pembelian
  3. Simulasi sejajar untuk Laporan Arus Kas
  4. Simulasi sejajar untuk Laporan Penggajian
  5. Simulasi sejajar untuk Laporan Persediaan

C. Lakukan ekspor data untuk setiap laporan yang benar dengan rincian sebagai berikut:
  1. Laporan Penjualan dan Laporan Pembelian diekspor ke dalam bentuk csv
  2. Laporan Kas, Laporan Gaji, dan Laporan Persediaan diekspor ke dalam bentuk txt (fixed width)

Jawaban silahkan dijadikan 1 folder dan di rar serta diupload di LMS untuk Pertemuan 7. 

### Selamat Mengerjakan!!
Saya standby di WA atau Zoom

In [ ]:
#@title Impor
import pandas as pd 
import datetime as dt
import random as rd
from itertools import chain
from google.colab import files

npm_awal = npm_saya
selisih_seed = int(str(npm_saya)[8])+1
npm_saya = int(str(npm_saya)[4:9])
current_inventory = pd.read_csv('https://raw.githubusercontent.com/zaresilkwood/studentdata/main/pertemuan6/current_inventory.csv', sep=';')
pegawai = pd.read_csv('https://raw.githubusercontent.com/zaresilkwood/studentdata/main/pertemuan6/pegawai.csv', sep=';')
neraca = pd.read_csv('https://raw.githubusercontent.com/zaresilkwood/studentdata/main/pertemuan6/referensi_data.csv', sep = ';')

In [ ]:
#@title Initialize
# In[3]:


#list id_pegawai
id_pegawai = list(pegawai['ID_PEGAWAI'])
len_id_pegawai = len(id_pegawai)-1

#list id_sales
id_sales = list(pegawai.loc[pegawai['PETUGAS'] == 'SALES', 'ID_PEGAWAI'])
len_id_sales = len(id_sales)-1

#list id_purchase
id_purchase = list(pegawai.loc[pegawai['PETUGAS'] == 'PURCHASE', 'ID_PEGAWAI'])
len_id_purchase = len(id_purchase)-1

#buat dictionary key value dari data barang
dict_barang = dict(zip(current_inventory['KD_BRG'], current_inventory['QTY'])) 
dict_beli = dict(zip(current_inventory['KD_BRG'], current_inventory['HARGA_BELI']))
dict_jual = dict(zip(current_inventory['KD_BRG'], current_inventory['HARGA_JUAL']))
id_barang = list(current_inventory['KD_BRG'])
len_barang = len(dict_barang)-1

#beragam diskon
discount = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.05, 0.1, 0.15, 0.2]
len_discount = len(discount)-1

#saldo
awal_kas = 239500000
current_kas = awal_kas
current_persediaan = sum([dict_beli[kode]*dict_barang[kode] for kode in id_barang])
current_ekuitas = awal_kas + current_persediaan
current_gaji = 0
current_penjualan = 0
current_potongan = 0
current_hpp = 0
current_utang = 0
current_piutang = 0 


# In[4]:


#inisiate data
pembayaran = ['kas', 'kas', 'kas', 'kas', 'kas', 'utang']
len_pembayaran = len(pembayaran)-1
penerimaan = ['kas', 'kas', 'kas', 'kas', 'kas', 'piutang']
len_penerimaan = len(penerimaan)-1
id_transaksi = 1

#tanggal awal
tanggal = dt.datetime(2020, 1, 1)
akhir_tahun = dt.datetime(2021, 1, 1)
bulan_gaji = 1
tahun_gaji = 2020
tanggal_gaji = dt.datetime(tahun_gaji, bulan_gaji, 20)

#tabel gaji
gaji_id = []
gaji_tanggal = []
gaji_id_pegawai = []
gaji_pokok = []
gaji_tunjangan = []

#tabel penjualan
sell_id = []
sell_tanggal = []
sell_id_pegawai = []
sell_kd_barang = []
sell_kuantitas = []
sell_discount = []

#tabel pembelian
buy_id = []
buy_tanggal = []
buy_id_pegawai = []
buy_kd_barang = []
buy_kuantitas = []

#tabel kas
id_kas = []
tanggal_kas = []
jenis_kas = []
jumlah_kas = []

#tabel utang
id_utang = []
tanggal_utang = []
jenis_utang = []
jumlah_utang = []

#tabel piutang
id_piutang =[]
tanggal_piutang = []
jenis_piutang = []
jumlah_piutang = []

#tabel pembayaran utang
lunas_utang_id = []
lunas_utang = []
lunas_utang_tgl = []

#tabel pembayaran piutang
lunas_piutang_id = []
lunas_piutang = []
lunas_piutang_tgl = []

In [ ]:
#@title Create Data
# ### Pembuatan Data Utama

# In[5]:


#pake for
#for i in range(200):

#pake while
while tanggal < akhir_tahun:
    
    #random seed
    rd.seed(npm_saya)
    
    #pengecekan tanggal utang apabila ada saldo utang
    if lunas_utang_tgl:
        if lunas_utang_tgl[0] >= tanggal:
            pass
        else:
            #tabel riwayat utang
            id_utang.append(lunas_utang_id[0])
            tanggal_utang.append(lunas_utang_tgl[0])
            jenis_utang.append('pelunasan utang')
            jumlah_utang.append(lunas_utang[0])
            
            #kas berkurang
            current_kas -= lunas_utang[0]
            
            #tabel kas
            id_kas.append(lunas_utang_id[0])
            tanggal_kas.append(lunas_utang_tgl[0])
            jenis_kas.append('pembayaran utang')
            jumlah_kas.append(lunas_utang[0])
            
            #hapus data pertama
            lunas_utang_id.pop(0)
            lunas_utang.pop(0)
            lunas_utang_tgl.pop(0)              
    else:
        pass

    #pengecekan tanggal piutang apabila ada saldo piutang
    if lunas_piutang_tgl:
        if lunas_piutang_tgl[0] >= tanggal:
            pass
        else:
            #tabel riwayat piutang
            id_piutang.append(lunas_piutang_id[0])
            tanggal_piutang.append(lunas_piutang_tgl[0])
            jenis_piutang.append('penerimaan piutang')
            jumlah_piutang.append(lunas_piutang[0])
            
            #kas bertambah
            current_kas += lunas_piutang[0]
            
            #tabel kas
            id_kas.append(lunas_piutang_id[0])
            tanggal_kas.append(lunas_piutang_tgl[0])
            jenis_kas.append('penerimaan piutang')
            jumlah_kas.append(lunas_piutang[0])
            
            #hapus data pertama
            lunas_piutang_id.pop(0)
            lunas_piutang.pop(0)
            lunas_piutang_tgl.pop(0)              
    else:
        pass
    
    #Pengecekan Gaji Bulanan Karyawan
    if tanggal_gaji >= tanggal:
        pass
        
    else:
        #bayar gaji karyawan di tanggal gaji
        gaji_id.append(['TRX-'+str(id_transaksi).zfill(3)] * (len_id_pegawai+1))
        gaji_tanggal.append([tanggal] * (len_id_pegawai+1))
        gaji_id_pegawai.append(list(pegawai['ID_PEGAWAI']))
        gaji_pokok.append(list(pegawai['GAJI POKOK']))
        tunjangan = rd.sample(range(15, 30), len_id_pegawai+1)
        tunjangan = [hari*150000 for hari in tunjangan]
        gaji_tunjangan.append(tunjangan)
        total_gaji = sum(list(pegawai['GAJI POKOK'])) + sum(tunjangan)
        
        #pembayaran kas
        id_kas.append('TRX-'+str(id_transaksi).zfill(3))
        tanggal_kas.append(tanggal)
        jenis_kas.append('pembayaran gaji')
        jumlah_kas.append(total_gaji)
        current_kas -= total_gaji
            
        #tanggal gaji bulan depan
        bulan_gaji += 1
        if bulan_gaji == 13:
            bulan_gaji = 1
            tahun_gaji += 1
        else:
            pass
        tanggal_gaji = dt.datetime(tahun_gaji, bulan_gaji, 20)
        id_transaksi += 1
    
    #MULAI DARI SINI!! 
    kd_barang = id_barang[rd.randint(0,len_barang)]
    kuantitas = rd.randint(15,25)
    
    #penjualan atau pembelian
    if dict_barang[kd_barang]<kuantitas: 
        
        #pembelian
        buy_id.append('TRX-'+str(id_transaksi).zfill(3))
        buy_tanggal.append(tanggal)
        buy_id_pegawai.append(id_purchase[rd.randint(0, len_id_purchase)])
        buy_kd_barang.append(kd_barang)
        jumlah_pembelian = rd.randint(75,100)
        buy_kuantitas.append(jumlah_pembelian)
        dict_barang[kd_barang] += jumlah_pembelian
        
        #cara pembayaran pembelian
        if current_kas > jumlah_pembelian* dict_beli[kd_barang] + 50000000:
            
            #kas atau utang
            cara_beli = pembayaran[rd.randint(0, len_pembayaran)]
            if cara_beli == 'kas':            
            
                #pembayaran kas
                id_kas.append('TRX-'+str(id_transaksi).zfill(3))
                tanggal_kas.append(tanggal)
                jenis_kas.append('pembayaran kas')
                jumlah_kas.append(jumlah_pembelian* dict_beli[kd_barang])
                current_kas -= jumlah_pembelian* dict_beli[kd_barang]
                
            else:#pembayaran via utang
                #tabel utang
                id_utang.append('TRX-'+str(id_transaksi).zfill(3))
                tanggal_utang.append(tanggal)
                jenis_utang.append('pembelian barang')
                jumlah_utang.append(jumlah_pembelian* dict_beli[kd_barang])

                #tabel riwayat utang
                lunas_utang_id.append('TRX-'+str(id_transaksi).zfill(3))
                lunas_utang.append(jumlah_pembelian* dict_beli[kd_barang])
                lunas_utang_tgl.append(tanggal + dt.timedelta(days = 28))                
                
            
        else: #pembayaran via utang
            #tabel utang
            id_utang.append('TRX-'+str(id_transaksi).zfill(3))
            tanggal_utang.append(tanggal)
            jenis_utang.append('pembelian barang')
            jumlah_utang.append(jumlah_pembelian* dict_beli[kd_barang])
            
            #tabel riwayat utang
            lunas_utang_id.append('TRX-'+str(id_transaksi).zfill(3))
            lunas_utang.append(jumlah_pembelian* dict_beli[kd_barang])
            lunas_utang_tgl.append(tanggal + dt.timedelta(days = 28))
        
        #tanggal bertambah antara 0 s.d. 2
        tanggal = tanggal + dt.timedelta(days = rd.randint(0,2))
        id_transaksi += 1
            
                
    else: #penjualan
        sell_id.append('TRX-'+str(id_transaksi).zfill(3))
        sell_tanggal.append(tanggal)
        sell_id_pegawai.append(id_sales[rd.randint(0,len_id_sales)])
        sell_kd_barang.append(kd_barang)
        sell_kuantitas.append(kuantitas)
        diskon = discount[rd.randint(0,len_discount)]
        sell_discount.append(diskon) #tambah diskon di bawah
        dict_barang[kd_barang] -= kuantitas
        
        #cara pembayaran penjualan
        cara_jual = penerimaan[rd.randint(0, len_penerimaan)]
        if cara_jual == 'kas':
            
            #tabel penjualan kas
            id_kas.append('TRX-'+str(id_transaksi).zfill(3))
            tanggal_kas.append(tanggal)
            jenis_kas.append('penjualan kas')
            jumlah_kas.append(dict_jual[kd_barang]*kuantitas - dict_jual[kd_barang]*kuantitas*diskon)
            current_kas += dict_jual[kd_barang]*kuantitas - dict_jual[kd_barang]*kuantitas*diskon
            
        else: #penjualan via piutang
            #tabel piutang
            id_piutang.append('TRX-'+str(id_transaksi).zfill(3))
            tanggal_piutang.append(tanggal)
            jenis_piutang.append('penjualan barang')
            jumlah_piutang.append(dict_jual[kd_barang]*kuantitas - dict_jual[kd_barang]*kuantitas*diskon)
            
            #tabel riwayat piutang
            lunas_piutang_id.append('TRX-'+str(id_transaksi).zfill(3))
            lunas_piutang.append(dict_jual[kd_barang]*kuantitas - dict_jual[kd_barang]*kuantitas*diskon)
            lunas_piutang_tgl.append(tanggal + dt.timedelta(days = 14))
        
        #tanggal bertambah antara 0 s.d. 2
        tanggal = tanggal + dt.timedelta(days = rd.randint(0,2))    
        id_transaksi += 1
        
    #ganti random seed
    npm_saya = npm_saya + selisih_seed
    


# ### Ganti ke Tabel

# In[6]:


#pembuatan tabel
#tabel gaji
gaji_id = list(chain(*gaji_id))
gaji_tanggal = list(chain(*gaji_tanggal))
gaji_id_pegawai = list(chain(*gaji_id_pegawai))
gaji_pokok = list(chain(*gaji_pokok))
gaji_tunjangan = list(chain(*gaji_tunjangan))

tabel_gaji = {
    'id_trx':gaji_id,
    'tanggal':gaji_tanggal,
    'id_pegawai':gaji_id_pegawai,
    'gaji_pokok':gaji_pokok,
    'tunjangan':gaji_tunjangan,
}
tabel_gaji = pd.DataFrame(tabel_gaji)

#tabel penjualan
tabel_penjualan = {
    'id_trx':sell_id,
    'tanggal':sell_tanggal,
    'id_pegawai':sell_id_pegawai,
    'kd_barang':sell_kd_barang,
    'kuantitas': sell_kuantitas,
    'discount':sell_discount
}
tabel_penjualan = pd.DataFrame(tabel_penjualan)

#tabel pembelian
tabel_pembelian = {
    'id_trx':buy_id,
    'tanggal':buy_tanggal,
    'id_pegawai':buy_id_pegawai,
    'kd_barang':buy_kd_barang,
    'kuantitas':buy_kuantitas
}
tabel_pembelian = pd.DataFrame(tabel_pembelian)

#tabel kas
tabel_kas = {
    'id_trx':id_kas,
    'tanggal':tanggal_kas,
    'jenis_transaksi':jenis_kas,
    'jumlah':jumlah_kas
}
tabel_kas = pd.DataFrame(tabel_kas)

#tabel utang
tabel_utang = {
    'id_trx':id_utang,
    'tanggal':tanggal_utang,
    'jenis_transaksi':jenis_utang,
    'jumlah':jumlah_utang
}
tabel_utang = pd.DataFrame(tabel_utang)

#tabel piutang
tabel_piutang = {
    'id_trx':id_piutang,
    'tanggal':tanggal_piutang,
    'jenis_transaksi':jenis_piutang,
    'jumlah':jumlah_piutang
}
tabel_piutang = pd.DataFrame(tabel_piutang)


# ### Editing Tabel

# In[7]:


number = list(range(1, len(tabel_gaji)+1))
number = [str(item).zfill(3) for item in number]
number = list(map(lambda x: 'PAY-' + x, number))
tabel_gaji.insert(0, 'kd_pembelian', number)


# In[8]:


tabel_kas["debit"] = tabel_kas.apply(lambda x: x['jumlah'] if (x['jenis_transaksi'] == 'penjualan kas') or
                                     (x['jenis_transaksi'] == 'penerimaan piutang') else 0, axis=1)
tabel_kas["kredit"] = tabel_kas.apply(lambda x: x['jumlah'] if (x['jenis_transaksi'] == 'pembayaran kas') or
                                      (x['jenis_transaksi'] == 'pembayaran utang') or
                                      (x['jenis_transaksi'] == 'pembayaran gaji') else 0, axis=1)
tabel_kas = tabel_kas.drop(columns=['jumlah'])


# In[9]:


tabel_utang["debit"] = tabel_utang.apply(lambda x: x['jumlah'] if x['jenis_transaksi'] == 'pelunasan utang' else 0, axis=1)
tabel_utang["credit"] = tabel_utang.apply(lambda x: x['jumlah'] if x['jenis_transaksi'] == 'pembelian barang' else 0, axis=1)
tabel_utang = tabel_utang.drop(columns=['jumlah'])
list_utang = tabel_utang.loc[tabel_utang['credit'] > 0]


# In[10]:


tabel_piutang["debit"] = tabel_piutang.apply(lambda x: x['jumlah'] if x['jenis_transaksi'] == 'penjualan barang' else 0, axis=1)
tabel_piutang["kredit"] = tabel_piutang.apply(lambda x: x['jumlah'] if x['jenis_transaksi'] == 'penerimaan piutang' else 0, axis=1)
tabel_piutang = tabel_piutang.drop(columns=['jumlah'])
list_piutang = tabel_piutang.loc[tabel_piutang['debit'] > 0]


# In[11]:


tabel_pembelian["harga"] = tabel_pembelian['kd_barang'].map(dict_beli)
tabel_pembelian["total"] = tabel_pembelian['kuantitas']*tabel_pembelian['harga']
tabel_pembelian = pd.merge(tabel_pembelian, tabel_kas[['id_trx', 'kredit']], how='left', on=['id_trx'])
tabel_pembelian = pd.merge(tabel_pembelian, list_utang[['id_trx', 'credit']], how='left', on=['id_trx'])
tabel_pembelian[['credit']] = tabel_pembelian[['credit']].fillna(0)
tabel_pembelian['credit'] = pd.to_numeric(tabel_pembelian['credit'], downcast='integer', errors='coerce')
tabel_pembelian["kas"] = tabel_pembelian.apply(lambda x: 0 if x['kredit'] == x['credit'] else x['kredit'], axis=1)
tabel_pembelian = tabel_pembelian.drop(columns=['kredit'])
tabel_pembelian = tabel_pembelian.rename(columns={'credit': 'utang'})
number = list(range(1, len(tabel_pembelian)+1))
number = [str(item).zfill(3) for item in number]
number = list(map(lambda x: 'BUY-' + x, number))
tabel_pembelian.insert(0, 'kd_pembelian', number)
tabel_pembelian[['kas']] = tabel_pembelian[['kas']].fillna(0)
tabel_pembelian[['utang']] = tabel_pembelian[['utang']].fillna(0)


# In[12]:


tabel_penjualan["harga"] = tabel_penjualan['kd_barang'].map(dict_jual)
tabel_penjualan["penjualan"] = tabel_penjualan['kuantitas']*tabel_penjualan['harga']
tabel_penjualan["diskon"] = tabel_penjualan['discount']*tabel_penjualan['penjualan']
tabel_penjualan["total"] = tabel_penjualan['penjualan']-tabel_penjualan['diskon']
tabel_penjualan = pd.merge(tabel_penjualan, tabel_kas[['id_trx', 'debit']], how='left', on=['id_trx'])
tabel_penjualan = pd.merge(tabel_penjualan, list_piutang[['id_trx', 'debit']], how='left', on=['id_trx'])
tabel_penjualan['kas'] = tabel_penjualan.apply(lambda x: 0 if x['debit_x'] == x['debit_y'] else x['debit_x'], axis=1)
tabel_penjualan = tabel_penjualan.rename(columns={'debit_y': 'piutang'})
tabel_penjualan = tabel_penjualan.drop(columns=['discount', 'debit_x'])
tabel_penjualan[['piutang']] = tabel_penjualan[['piutang']].fillna(0)
tabel_penjualan[['kas']] = tabel_penjualan[['kas']].fillna(0)
for column_name in ['diskon', 'total', 'piutang', 'kas']:
    tabel_penjualan[column_name] = tabel_penjualan[column_name].astype('Int64')
number = list(range(1, len(tabel_penjualan)+1))
number = [str(item).zfill(3) for item in number]
number = list(map(lambda x: 'SLS-' + x, number))
tabel_penjualan.insert(0, 'kd_penjualan', number)


# ### Create Challenge
# 
# Data Validation
# 1. Otorisator yang salah
# 2. Kesalahan kode pembelian dan kode penjualan
# 3. Kesalahan periode
# 4. Data kosong di harga
# 5. Data integer jadi text
# 6. Adanya nilai minus
# 7. Kesalahan harga
# 8. Perhitungan yang salah
# 
# Report Check -- buat laporannya dulu
# 1. Laporan Penjualan salah
# 2. Laporan Pembelian salah
# 3. Laporan Persediaan salah
# 4. Laporan Arus Kas 
# 5. Laporan Beban Gaji salah
# 6. Neraca salah

# ## Pembuatan Laporan

# ### Laporan

# #### Laporan Penjualan

# In[13]:


laporan_penjualan = pd.pivot_table(tabel_penjualan, columns=['id_pegawai'], index=['kd_barang'], values='total', aggfunc='sum').style
laporan_penjualan = laporan_penjualan.data
laporan_penjualan = laporan_penjualan.reset_index()
laporan_penjualan


# #### Laporan Pembelian

# In[14]:


dict_bulan = {
    '1':'Januari',
    '2':'Februari',
    '3':'Maret',
    '4':'April',
    '5':'Mei',
    '6':'Januari',
    '7':'Februari',
    '8':'Maret',
    '9':'April',
    '10':'Mei',
    '11':'November',
    '12':'Desember'
}


# In[15]:


tabel_pembelian_lap = tabel_pembelian.copy()
tabel_pembelian_lap["bulan"] = pd.DatetimeIndex(tabel_pembelian_lap['tanggal']).month
tabel_pembelian_lap['bulan'] = tabel_pembelian_lap['bulan'].astype('string')
laporan_pembelian = tabel_pembelian_lap.groupby(['kd_barang']).agg(utang=('utang', 'sum'), kas=('kas', 'sum')).reset_index()
for column_name in ['utang', 'kas']:
    laporan_pembelian[column_name] = laporan_pembelian[column_name].astype('int')
laporan_pembelian


# #### Laporan Persediaan
# 

# In[16]:


persediaan_jual_qty = tabel_penjualan[['kd_barang', 'kuantitas']].groupby(['kd_barang'], group_keys=False).sum()
persediaan_jual_qty = persediaan_jual_qty.reset_index()

persediaan_beli_qty = tabel_pembelian[['kd_barang', 'kuantitas']].groupby(['kd_barang'], group_keys=False).sum()
persediaan_beli_qty = persediaan_beli_qty.reset_index()

current_inventory = current_inventory.rename(columns={'KD_BRG': 'kd_barang', 'QTY': 'kuantitas_awal'})
current_inventory["nilai_awal"] = current_inventory['kuantitas_awal']*current_inventory['HARGA_BELI']
laporan_persediaan = pd.merge(current_inventory, persediaan_beli_qty[['kd_barang', 'kuantitas']], how='inner', on=['kd_barang'])
laporan_persediaan["nilai_pembelian"] = laporan_persediaan['HARGA_BELI']*laporan_persediaan['kuantitas']
laporan_persediaan = pd.merge(laporan_persediaan, persediaan_jual_qty[['kd_barang', 'kuantitas']], how='inner', on=['kd_barang'])
laporan_persediaan["nilai_hpp"] = laporan_persediaan['HARGA_BELI']*laporan_persediaan['kuantitas_y']
laporan_persediaan = laporan_persediaan.rename(columns={'kuantitas_x': 'kuantitas_pembelian', 'kuantitas_y': 'kuantitas_hpp'})
laporan_persediaan["kuantitas_akhir"] = laporan_persediaan['kuantitas_awal']+laporan_persediaan['kuantitas_pembelian']-laporan_persediaan['kuantitas_hpp']
laporan_persediaan["nilai_akhir"] = laporan_persediaan['nilai_awal']+laporan_persediaan['nilai_pembelian']-laporan_persediaan['nilai_hpp']
laporan_persediaan = laporan_persediaan.drop(columns=['HARGA_BELI', 'HARGA_JUAL'])


# #### Laporan Arus Kas

# In[17]:


laporan_kas = tabel_kas.groupby(['jenis_transaksi']).agg(debit_sum=('debit', 'sum'), kredit_sum=('kredit', 'sum')).reset_index()
laporan_kas.loc[-1] = ['saldo awal', awal_kas, 0]
laporan_kas.index = laporan_kas.index + 1
laporan_kas.sort_index(inplace = True)
laporan_kas


# #### Laporan Beban Gaji

# In[18]:


tabel_gaji_laporan = tabel_gaji.copy()
tabel_gaji_laporan["bulan"] = pd.DatetimeIndex(tabel_gaji_laporan['tanggal']).month
tabel_gaji_laporan = tabel_gaji_laporan.groupby(['bulan']).agg(gaji_pokok_sum=('gaji_pokok', 'sum'), tunjangan_sum=('tunjangan', 'sum')).reset_index()
tabel_gaji_laporan['bulan'] = tabel_gaji_laporan['bulan'].astype('string')
tabel_gaji_laporan["Bulan"] = tabel_gaji_laporan['bulan'].map(dict_bulan)
tabel_gaji_laporan = tabel_gaji_laporan.rename(columns={'gaji_pokok_sum': 'gaji_pokok', 'tunjangan_sum': 'tunjangan'})
cols = ['Bulan', 'gaji_pokok', 'tunjangan']
tabel_gaji_laporan = tabel_gaji_laporan[cols]
tabel_gaji_laporan


# #### Laporan Neraca

# In[19]:


KAS = [sum(tabel_penjualan['kas'])+awal_kas, sum(tabel_pembelian['kas'])+sum(tabel_gaji['gaji_pokok'])+sum(tabel_gaji['tunjangan'])]
PIUTANG = [sum(tabel_penjualan['piutang']), None]
PERSEDIAAN = [sum(laporan_persediaan['nilai_awal'])+sum(laporan_persediaan['nilai_pembelian']),
              sum(laporan_persediaan['nilai_hpp'])]
UTANG = [None, sum(tabel_pembelian['utang'])]
EKUITAS = [None, current_ekuitas]
PENJUALAN = [None, sum(tabel_penjualan['penjualan'])]
POTONGAN_PENJUALAN = [sum(tabel_penjualan['diskon']), None]
HPP = [sum(laporan_persediaan['nilai_hpp']), None]
BEBAN_GAJI = [sum(tabel_gaji['gaji_pokok'])+sum(tabel_gaji['tunjangan']), None]
data2020 = [KAS, PIUTANG, PERSEDIAAN, UTANG, EKUITAS, PENJUALAN, POTONGAN_PENJUALAN, HPP, BEBAN_GAJI]
data2020 = pd.DataFrame(data2020, columns = ['debit2020', 'kredit2020'])


# In[20]:


#Load format neraca

neraca.iloc[neraca['nama_akun']=='KAS', 2] = awal_kas
neraca.iloc[neraca['nama_akun']=='PERSEDIAAN', 2] = current_persediaan
neraca.iloc[neraca['nama_akun']=='EKUITAS', 3] = current_ekuitas
neraca = pd.concat([neraca.reset_index(drop=True), data2020.reset_index(drop=True)], axis=1)
neraca = neraca.fillna(0)
neraca['debit2019'] = pd.to_numeric(neraca['debit2019'], downcast='integer', errors='coerce')
neraca['kredit2019'] = pd.to_numeric(neraca['kredit2019'], downcast='integer', errors='coerce')
neraca['debit2020'] = pd.to_numeric(neraca['debit2020'], downcast='integer', errors='coerce')
neraca['kredit2020'] = pd.to_numeric(neraca['kredit2020'], downcast='integer', errors='coerce')

#@title Challenge
# ## Buat Kesalahan

# ### Kesalahan Data Tabel Penjualan

# #### Otorisator yang salah (penjualan dan pembelian)

# In[21]:


tabel_penjualan_salah = tabel_penjualan.copy()

#tabel penjualan
for i in range(3):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    pegawai_salah = rd.choice(id_purchase)
    tabel_penjualan_salah.iloc[baris, 3] = pegawai_salah
    npm_saya += 3


# #### Double Kode Penjualan

# In[56]:


#tabel penjualan
for i in range(4):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(2,panjang_salah-2)
    tabel_penjualan_salah.iloc[baris, 0] = tabel_penjualan_salah.iloc[baris-1, 0] 
    npm_saya += 5

# #### Kesalahan kode penjualan
# 

# In[22]:


#tabel penjualan
for i in range(3):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 0] = tabel_penjualan_salah.iloc[baris, 0][2:]
    npm_saya += 3


# #### Kesalahan periode pada tabel penjualan
# 

# In[35]:


#tabel penjualan
bulan_salah = [-1, 1]
for i in range(4):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    bulan_salah_p = rd.choice(bulan_salah)
    tabel_penjualan_salah.iloc[baris, 2] = tabel_penjualan_salah.iloc[baris, 2] + pd.DateOffset(years = 1)*bulan_salah_p
    npm_saya += 3


# #### Data kosong di harga atau kuantitas penjualan
# 

# In[41]:


for i in range(4):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 6] = ''
    npm_saya += 4
    
for i in range(3):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 5] = ''
    npm_saya += 4


# #### kode barang tidak ada di referensi
# 

# In[43]:


barang_aneh = ['SLTKR', 'LSP02', 'SQ231', 'SDKGS', 'YUTWS']
for i in range(3):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    brg_aneh = rd.choice(barang_aneh)
    tabel_penjualan_salah.iloc[baris, 4] = brg_aneh
    npm_saya += 5


# #### Adanya nilai minus
# 

# In[46]:


for i in range(2):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 7] = tabel_penjualan_salah.iloc[baris, 7]*-1
    npm_saya += 6
    
for i in range(2):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 9] = tabel_penjualan_salah.iloc[baris, 9]*-1
    npm_saya += 6


# #### Kesalahan harga

# In[49]:


harga_salah = [1300000, 1400000, 850000, 1440000]
for i in range(4):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    hrg_salah = rd.choice(harga_salah)
    tabel_penjualan_salah.iloc[baris, 6] = hrg_salah
    npm_saya += 6


# #### Perhitungan yang salah

# In[53]:


for i in range(3):
    rd.seed(npm_saya)
    panjang_salah = len(tabel_penjualan_salah)-1
    baris = rd.randint(0,panjang_salah)
    npm_saya += 2
    rd.seed(npm_saya)
    baris2 = rd.randint(0,panjang_salah)
    tabel_penjualan_salah.iloc[baris, 9] = tabel_penjualan_salah.iloc[baris2, 9]
    npm_saya += 7


# ### Kesalahan Laporan

# #### Kesalahan Laporan Penjualan

# In[37]:


laporan_penjualan_salah = laporan_penjualan.copy()
pilihan_penjualan = list(laporan_penjualan_salah.columns)[1:4]
for i in range(3):
    rd.seed(npm_saya)
    kolom = rd.randint(1,2)
    baris = rd.randint(0,9)
    nilai = rd.randint(350000, 1650000)*500
    laporan_penjualan_salah.iloc[baris,kolom] = nilai
    npm_saya += 2
laporan_penjualan_salah = laporan_penjualan_salah.reset_index()


# #### Kesalahan Laporan Pembelian

# In[26]:


laporan_pembelian_salah = laporan_pembelian.copy()
pilihan_pembelian = ['utang', 'kas']
for i in range(3):
    rd.seed(npm_saya)
    kolom = rd.choice(pilihan_pembelian)
    baris = rd.randint(0,9)
    nilai = rd.randint(200000, 1200000)*500
    laporan_pembelian_salah.iloc[baris,laporan_pembelian_salah.columns.get_loc(kolom)] = nilai
    npm_saya += 2


# #### Kesalahan Laporan Persediaan

# In[27]:


#kesalahan laporan persediaan
persediaan_salah = laporan_persediaan.copy()
dict_salah = {
    'kuantitas_pembelian':'nilai_pembelian',
    'kuantitas_hpp':'nilai_hpp'
}

#loop
for i in range(3):
    rd.seed(npm_saya)
    #barang yang salah
    sample_barang = rd.sample(id_barang, 1)
    sample_barang = ''.join(sample_barang)
    #kolom yang salah
    kolom_salah = rd.sample(dict_salah.keys(), 1)
    kolom_salah = ''.join(kolom_salah)
    #mengganti nilai
    qty_salah = persediaan_salah.loc[persediaan_salah['kd_barang'] == sample_barang, kolom_salah]
    qty_salah -= rd.randint(0,30)
    persediaan_salah.loc[persediaan_salah['kd_barang'] == sample_barang, kolom_salah] = qty_salah
    persediaan_salah.loc[persediaan_salah['kd_barang'] == sample_barang, dict_salah[kolom_salah]] = qty_salah*dict_beli[sample_barang]
    #penyesuaian kuantitas dan nilai akhir
    persediaan_salah["kuantitas_akhir"] = persediaan_salah['kuantitas_awal']+persediaan_salah['kuantitas_pembelian']-persediaan_salah['kuantitas_hpp']
    persediaan_salah["nilai_akhir"] = persediaan_salah['nilai_awal']+persediaan_salah['nilai_pembelian']-persediaan_salah['nilai_hpp']
    #ganti seed
    npm_saya = npm_saya*3


# #### Kesalahan Laporan Arus Kas

# In[28]:


#TIDAK ADA!!!!


# #### Kesalahan Laporan Beban Gaji

# In[29]:


laporan_gaji_salah = tabel_gaji_laporan.copy()
for i in range(2):
    rd.seed(npm_saya)
    baris = rd.randint(0,11)
    nilai = rd.randint(720, 820)*50000
    laporan_gaji_salah.iloc[baris,2] = nilai
    npm_saya += 2


# #### Kesalahan Neraca

# In[30]:


neraca_edit = neraca.copy()
wrong_debit = [0,1,2,6,7,8]
len_wrong_debit = len(wrong_debit)-1
wrong_credit = [0,2,3,4,5]
len_wrong_credit = len(wrong_credit)-1
for i in range(2):
    rd.seed(npm_saya)
    seed_debit = rd.randint(0,len_wrong_debit)
    seed_credit = rd.randint(0,len_wrong_credit)
    #debit 4 credit 5
    nilai_penambah = rd.randint(2,90)*1000
    neraca_edit.iloc[wrong_debit[seed_debit],4] = neraca_edit.iloc[wrong_debit[seed_debit],4]+nilai_penambah
    neraca_edit.iloc[wrong_credit[seed_credit],5] = neraca_edit.iloc[wrong_credit[seed_credit],5]+nilai_penambah
    wrong_debit.pop(seed_debit)
    wrong_credit.pop(seed_credit)
    len_wrong_debit = len(wrong_debit)-1
    len_wrong_credit = len(wrong_credit)-1

#@title Download
!pip install xlsxwriter
import xlsxwriter
nama_soal1 = 'soal_untuk_'+str(npm_awal)+'_validasi.xlsx'
with pd.ExcelWriter(nama_soal1, engine='xlsxwriter', date_format='yyyy/mm/dd') as writer:    
    tabel_penjualan_salah.to_excel(writer, sheet_name='penjualan', index=False)
    pegawai.to_excel(writer, sheet_name='pegawai', index = False)
    current_inventory.to_excel(writer, sheet_name='persediaan', index=False)

files.download('soal_untuk_'+str(npm_awal)+'_validasi.xlsx')

nama_soal2 = 'soal_untuk_'+str(npm_awal)+'_simulasi_sejajar.xlsx'
with pd.ExcelWriter(nama_soal2, engine='xlsxwriter', date_format='yyyy/mm/dd') as writer:    
    tabel_pembelian.to_excel(writer, sheet_name='pembelian', index=False)
    tabel_penjualan.to_excel(writer, sheet_name='penjualan', index=False)
    tabel_kas.to_excel(writer, sheet_name='kas', index=False)
    tabel_gaji.to_excel(writer, sheet_name='penggajian', index=False)
    current_inventory.to_excel(writer, sheet_name='persediaan_awal', index=False)
    
    laporan_penjualan_salah.to_excel(writer, sheet_name='lap_penjualan', index=False)
    laporan_pembelian_salah.to_excel(writer, sheet_name='lap_pembelian', index=False)
    laporan_kas.to_excel(writer, sheet_name='lap_kas', index=False)
    laporan_gaji_salah.to_excel(writer, sheet_name='lap_gaji', index=False)
    persediaan_salah.to_excel(writer, sheet_name='lap_persediaan', index=False)

files.download('soal_untuk_'+str(npm_awal)+'_simulasi_sejajar.xlsx')